In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
#loader=PyPDFLoader(['syllabus.pdf','Machine Learning for Beginners'])
file_paths = "C:\\Users\\user\\Agentic _AI_workspace\\2-LangchainConti\\2.2-DataTransformer"
docs=[]
for file in os.listdir(file_paths):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(file_paths, file)
            loader = PyPDFLoader(pdf_path)
            docs.extend(loader.load())


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_spliter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_document=text_spliter.split_documents(docs)

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()
vectorstore=FAISS.from_documents(docs,embeddings)
vectorstore

C:\Users\user\AppData\Local\Temp\ipykernel_17232\926516400.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings=OpenAIEmbeddings()


In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

## Langsmith Tracking and tracing

os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [6]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-4")
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x000001C409307E90> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001C409304650> root_client=<openai.OpenAI object at 0x000001C40810C530> root_async_client=<openai.AsyncOpenAI object at 0x000001C408127F50> model_name='gpt-4' model_kwargs={} openai_api_key=SecretStr('**********')


In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

In [8]:
retriever=vectorstore.as_retriever()

from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001C409307E90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001C409304650>, root_client=<openai.OpenAI object at 0x000001C40810C530>, root_async_client=<openai.AsyncOpenAI object at 0x000001C408127F50>, model_name='gpt-4', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_doc

In [9]:
from langchain.chains import create_retrieval_chain

retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C40A105A60>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | 

In [10]:
Result=retrieval_chain.invoke({"input":"The module also explores the Transformer architecture"})

In [11]:
query="The module will also guide you through writing simple functions "
r=vectorstore.similarity_search(query)
r

[Document(id='4c8d9939-d1aa-4754-81a7-bb7d81f054a2', metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-01-30T20:27:03+00:00', 'title': 'Ultimate Data Science & GenAI Bootcamp', 'moddate': '2025-01-30T20:26:59+00:00', 'keywords': 'DAGdmhcqnYw,BAEmsmap8Lg,0', 'author': 'monal singh', 'containsaigeneratedcontent': 'Yes', 'source': 'C:\\Users\\user\\Agentic _AI_workspace\\2-LangchainConti\\2.2-DataTransformer\\syllabus.pdf', 'total_pages': 34, 'page': 4, 'page_label': '5'}, page_content="In this module, you’ll get a solid introduction to Python, covering essential programming concepts\nsuch as variables, data types, operators, and control flow. You’ll learn how to manipulate strings,\nlists, dictionaries, and other basic data structures. The module will also guide you through writing\nsimple functions and using loops and conditionals effectively. By the end, you'll have a strong\nunderstanding of Python syntax, preparing you to tackle more complex programming challen

In [12]:
retriver=vectorstore.as_retriever()

docs=retriever.invoke(query)
docs[0].page_content

"In this module, you’ll get a solid introduction to Python, covering essential programming concepts\nsuch as variables, data types, operators, and control flow. You’ll learn how to manipulate strings,\nlists, dictionaries, and other basic data structures. The module will also guide you through writing\nsimple functions and using loops and conditionals effectively. By the end, you'll have a strong\nunderstanding of Python syntax, preparing you to tackle more complex programming challenges\nand form a foundation for learning advanced concepts.\nUltimate Data Science & GenAI Bootcamp       Page  5\nPython Foundations\nModule 1\nTopics\nIntroduction to Python Comparison with other programming\nlanguages, Python objects: Numbers,\nBooleans, and Strings\nData Structures & Operations Container objects and mutability,\nOperators, Operator precedence and\nassociativity\nControl Flow Conditional statements, Loops, break\nand continue statements\nString Manipulation Basics of string objects, Inbu

In [13]:
import streamlit as st
st.set_page_config(page_title="Doc Searcher", page_icon=":robot:")
st.header("Query PDF Source")

form_input = st.text_input('Enter Query')
submit = st.button("Generate")

2025-02-16 14:21:38.877 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-16 14:21:38.885 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-16 14:21:41.090 
  command:

    streamlit run c:\Users\user\Agentic _AI_workspace\venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-02-16 14:21:41.092 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-16 14:21:41.097 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-16 14:21:41.101 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-16 14:21:41.105 Thread 'MainThread': missing ScriptRunContext! This warning can be i